In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import json
import random

In [9]:
filter_list = ['twitter.com', 'facebook.com', 'instagram.com', 'flickr.com']

eot_good_all = json.load(open('data/eot_good_all.json'))
print("Before", len(eot_good_all))
eot_good_all = [e for e in eot_good_all if not any([f in e['live_url'] for f in filter_list])]
print("After", len(eot_good_all))
eot_300 = random.sample(eot_good_all, 500)
json.dump(eot_300, open('ground-truth/ground_truth_eot_500.json', 'w'), indent=2)

Before 4093
After 3786


### Merge determinism results

In [12]:
num = 8
results = []
for i in range(num):
    data = json.load(open(f'ground-truth/determinism_results_{i}.json'))
    results.extend(data)
print(len(results))
json.dump(results, open('ground-truth/determinism_results.json', 'w'), indent=2)

487
